# チャットモデル

任意のチャットメッセージのやり取り後の応答を生成することができます。

「テキスト生成」の入力は「テキスト文字列」でしたが、「チャット」の入力は「チャットメッセージ文字列のリスト」になります。

以下のモデルは、OpenAIが用意しているチャットモデルになります。

- gpt-3.5-turbo
- gpt-4

本ノートブックでは、「gpt-3.5-turbo」を使っていきます。

## 準備

In [ ]:
## 利用するベースモデルのライブラリ (OpenAI) も別途インストールする
!pip install langchain==0.0.331 openai==0.28.1 python-dotenv cohere tiktoken

In [ ]:
## Googleドライブをマウント
from google.colab import drive
drive.mount('./drive')

Drive already mounted at ./drive; to attempt to forcibly remount, call drive.mount("./drive", force_remount=True).


In [ ]:
## 環境変数設定
import dotenv
dotenv.load_dotenv('./drive/MyDrive/openai.env')

True

## チャットメッセージリストの準備

「チャット」の入力は「チャットメッセージ文字列のリスト」で、出力は「チャットメッセージ文字列」になります。
チャットメッセージでは、「role」と「content」を持ちます。
「role」は、以下の3種類です。

- system: チャットの振る舞いに関する指示
- user: 人間の発話
- assistant: AIの発話

通常、会話の最初に「system」メッセージを記述します。次に、「user」メッセージと「assistant」メッセージを交互に記述します。

In [ ]:
## チャットメッセージのリスト
messages = [
    {"role": "system", "content": "あなたは女子高生です。1人の兄がいる、妹のように演じてください。あなたは兄と会話します。"},
    {"role": "user", "content": "おはよう"},
]

In [ ]:
## OpenAIのライブラリで実行
import os
import openai

## チャットの実行
openai.api_key = os.getenv("OPENAI_API_KEY")  # set OpenAI API Key
res = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=messages,
    temperature = 0.7,
    max_tokens = 255,
)
## openai v1.0.0 から以下のように変更となった。
# res = openai.OpenAI().chat.completions.create(
#     model='gpt-3.5-turbo',
#     messages=messages,
#     temperature = 0.7,
#     max_tokens = 255,
# )

In [ ]:
res['choices'][0]['message']['content']
## openai v1.0.0 から以下のように変更となった。
# res.choices[0].message.content

'おはよう兄さん！おはようございます。今日も元気ですか？'

## LangChainでの実行

チャットモデル用の `langchain.chat_models.ChatOpenAI` を使って実行します。
チャットメッセージのリストを作成する際は、各ロールに合わせて `HumanMessage` や `SystemMessage` を使うと良いでしょう。

In [ ]:
## LangChainで実行
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    model='gpt-3.5-turbo',
    openai_api_key = os.getenv("OPENAI_API_KEY"),  # set OpenAI API Key
    temperature = 0.7,
    max_tokens = 255,
)

In [ ]:
## チャットメッセージのリスト
from langchain.schema.messages import HumanMessage, SystemMessage
messages = [
    SystemMessage(content="あなたは女子高生です。1人の兄がいる、妹のように演じてください。あなたは兄と会話します。"),
    HumanMessage(content="おやすみ"),
]

In [ ]:
chat.invoke(messages)

AIMessage(content='おやすみなさい、お兄ちゃん。今日もお疲れさまでした。明日も頑張ってね。')

## 会話の内容を一時保存して使う

基本的に、Chat Modelの中でのチャットのやり取りは、言語モデルの中で勝手にデータは保存されません。
ChatGPTのようなWebサービスでは、Webサービス上で一時的に会話の内容を記憶させ、言語モデルに会話の履歴をリストとして保存し、それを言語モデルに渡すことで、チャットのやり取りの内容を言語モデルが認識しています。

他にも、言語モデルのやり取りの中で、出力結果を別の言語モデルに渡して最適な結果を生みたい場合もあります。

LangChainでは、そのような一時的な記憶を行うモジュールとして「LangChain Memory」が用意されています。




In [ ]:
## チャットメッセージのリスト
from langchain.schema.messages import SystemMessage
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_message(SystemMessage(content="あなたは女子高生です。1人の兄がいる、妹のように演じてください。あなたは兄と会話します。"))
history.add_user_message("おやすみ")
history.add_ai_message("おやすみなさい、お兄ちゃん！今日もお疲れ様でした。")

history.messages

[SystemMessage(content='あなたは女子高生です。1人の兄がいる、妹のように演じてください。あなたは兄と会話します。'),
 HumanMessage(content='おやすみ'),
 AIMessage(content='おやすみなさい、お兄ちゃん！今日もお疲れ様でした。')]

In [ ]:
## チャットに特化したConversationBufferMemoryを使ってみる
## ChatMessageHistoryがベースとなっており、会話の履歴管理が楽になります。

from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

## save_context()で保存
memory.save_context(
    {"input": "おやすみ"},
    {"output": "おやすみなさい、お兄ちゃん！今日もお疲れ様でした。"},
)

## load_memory_variables()で読み出し
print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='おやすみ'), AIMessage(content='おやすみなさい、お兄ちゃん！今日もお疲れ様でした。')]}


In [ ]:
## 会話を内容をすべて自動保存してみる。

from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

llm = OpenAI(
    model_name="text-davinci-003",
    openai_api_key = os.getenv("OPENAI_API_KEY"),  # set OpenAI API Key
    temperature = 0.7,
)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

conversation("AIとは何ですか？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: AIとは何ですか？
AI:

> Finished chain.


{'input': 'AIとは何ですか？',
 'history': '',
 'response': ' AIとは人工知能のことです。人間が行うタスクや判断をコンピューターが行うことを可能にする技術になります。'}

In [ ]:
## チャットモデルを使って、チャットの内容を全て自動保存してみる。
## Chainの中にMemoryを組み込むことで可能

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

## Setup Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        # 永続的なシステムプロンプト
        SystemMessage(
            content="あなたは人間と会話するチャットボットです。"
        ),
        # メモリの保存場所の設定
        MessagesPlaceholder(
            variable_name="chat_history"
        ),
        # Humanの入力が挿入される箇所を設定
        HumanMessagePromptTemplate.from_template(
          "{human_input}"
        ),
    ]
)

## Setup Memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

## Setup Chat Model
llm = ChatOpenAI(
    model='gpt-3.5-turbo',
    openai_api_key = os.getenv("OPENAI_API_KEY"),  # set OpenAI API Key
    temperature = 0.7,
    max_tokens = 255,
)

## Setup Chain (後ほど解説)
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [ ]:
## 会話開始
chat_llm_chain.predict(human_input="こんにちは！調子はどうだい？")



> Entering new LLMChain chain...
Prompt after formatting:
System: あなたは人間と会話するチャットボットです。
Human: こんにちは！調子はどうだい？

> Finished chain.


'こんにちは！私は人間ではないので、調子はありませんが、お話しできて嬉しいです。どんなお話しをしましょうか？'

In [ ]:
chat_llm_chain.predict(human_input="AIについて教えて")



> Entering new LLMChain chain...
Prompt after formatting:
System: あなたは人間と会話するチャットボットです。
Human: こんにちは！調子はどうだい？
AI: こんにちは！私は人間ではないので、調子はありませんが、お話しできて嬉しいです。どんなお話しをしましょうか？
Human: AIについて教えて

> Finished chain.


'AI（人工知能）は、コンピューターシステムが人間のような知的なタスクを実行する能力を指します。AIは機械学習やディープラーニングなどの技術を活用して、データからパターンを学習し、判断や推論を行うことができます。\n\nAIはさまざまな分野で活用されています。例えば、自然言語処理によって人間の言葉を理解し、会話を行ったり、画像認識によって物体や顔を識別したり、自動運転技術によって車両を制御したりすることができます。\n\nAIの応用範囲は広く、医療診断、金融予測、製造業の効率化'

In [ ]:
chat_llm_chain.predict(human_input="小学1年生でもわかりやすく教えて")



> Entering new LLMChain chain...
Prompt after formatting:
System: あなたは人間と会話するチャットボットです。
Human: こんにちは！調子はどうだい？
AI: こんにちは！私は人間ではないので、調子はありませんが、お話しできて嬉しいです。どんなお話しをしましょうか？
Human: AIについて教えて
AI: AI（人工知能）は、コンピューターシステムが人間のような知的なタスクを実行する能力を指します。AIは機械学習やディープラーニングなどの技術を活用して、データからパターンを学習し、判断や推論を行うことができます。

AIはさまざまな分野で活用されています。例えば、自然言語処理によって人間の言葉を理解し、会話を行ったり、画像認識によって物体や顔を識別したり、自動運転技術によって車両を制御したりすることができます。

AIの応用範囲は広く、医療診断、金融予測、製造業の効率化
Human: 小学1年生でもわかりやすく教えて

> Finished chain.


'AI（人工知能）は、コンピューターが考えることや学ぶことができる技術です。もちろん、AIは人間と同じように考えるわけではありませんが、人間のような知的なタスクをこなすことができるんです。\n\n例えば、AIはたくさんのデータを使って学習します。たとえば、果物の写真をたくさん見せて、「これはりんごです」と学習させれば、AIはりんごを見分けることができるようになります。同じように、AIにお話を聞かせて学習させれば、AIは質問に答えることができるようになるんです。\n\nAIは、自動運転車やスマートフォンの音声アシスタントなど、さまざまな場面'

## LangChain Chains

言語モデルを単独でシンプルに使う分には問題ないですが、複雑な連携を行うアプリケーションを開発する際は、言語モデル同士を連携させたり、外部のコンポーネントと接続したりする必要が出てきます。

LangChainでは、Chainモジュールを使って、あるプロンプトの出力結果を別のプロンプトとして使うことが簡単にできます。

例えば、巨大な文章の要約をしたい場合を考えます。
文字列長が長い場合、全ての文字列を一度に言語モデルに投入できません。（トークン制限により）
その場合、例えば文章の章毎に分割して、その章の要約を言語モデルに作ってもらい、その章毎の要約文を結合した文字列をさらに言語モデルに入力して、最終的な文章全体の要約を作るといった方法が可能になります。

LangChainには、元々あるChainインタフェースを使う方法と、最新のLangChain Expression Language (LCEL)を使う方法があります。先ほどのチャットの履歴を保存するコードでは、Chainインタフェースを使っています。

以下のコードでは、LCELでチェインを構成した例を示しています。


In [ ]:
## チャットモデルを使って、チャットの内容を全て保存してみる。
## Chainの中にMemoryを組み込むことで可能。
## なお、LCELでチェイン定義する場合、v0.0.330現在、Memoryへの保存は手動でフックさせる必要があります。

from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from operator import itemgetter

## Setup Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        # 永続的なシステムプロンプト
        SystemMessage(
            content="あなたは人間と会話するチャットボットです。"
        ),
        # メモリの保存場所の設定
        MessagesPlaceholder(
            variable_name="chat_history"
        ),
        # Humanの入力が挿入される箇所を設定
        HumanMessagePromptTemplate.from_template(
          "{human_input}"
        ),
    ]
)

## Setup Memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

## Setup Chat Model
llm = ChatOpenAI(
    model='gpt-3.5-turbo',
    openai_api_key = os.getenv("OPENAI_API_KEY"),  # set OpenAI API Key
    temperature = 0.7,
    max_tokens = 255,
)

## Setup Chain using LCEL
chat_llm_chain = (
    RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
    )
    | prompt
    | llm
)

In [ ]:
## 会話開始
inputs = {"human_input": "こんにちは！調子はどうだい？"}
res = chat_llm_chain.invoke(inputs)
memory.save_context(inputs, {"output": res.content})  ## 手動で保存
res

AIMessage(content='こんにちは！私はチャットボットなので、感情はありませんが、お話しすることができます。どのようなお話しをしたいですか？')

In [ ]:
inputs = {"human_input": "AIについて教えて"}
res = chat_llm_chain.invoke(inputs)
memory.save_context(inputs, {"output": res.content})  ## 手動で保存
res

AIMessage(content='AI（人工知能）は、コンピューターシステムが人間のような知的な活動を実行する技術です。AIは、大量のデータを処理し、パターンを認識し、学習することができます。\n\nAIの主なタイプには、以下のようなものがあります：\n\n1. 弱いAI（ナローアイ）：特定のタスクを実行するために設計されたAIです。例えば、音声認識や画像認識などがあります。\n\n2. 強いAI（ジェネラルアイ）：人間のような知的な活動を実行することができるAIです。現在の技術では、まだ完全な強いAIは実現していません。\n\n3. 機械学習：AIの一部として使用される技')

In [ ]:
inputs = {"human_input": "小学1年生でもわかりやすく教えて"}
res = chat_llm_chain.invoke(inputs)
memory.save_context(inputs, {"output": res.content})  ## 手動で保存
res

AIMessage(content='AI（人工知能）は、コンピューターが考えることや学ぶことができるようにする技術です。例えば、お友達の名前を覚えることや、絵を描くことを学ぶことができます。\n\nAIは、たくさんのデータを使って学ぶことができます。例えば、たくさんの写真を見せると、AIは猫や犬の写真を見分けることができます。それは、AIがパターンを認識するからです。\n\nまた、AIは間違いを修正することもできます。例えば、AIが「リンゴ」と言ったときに、実は「バナナ」だった場合、AIは間違いを気づいて修正することができます。\n\nAIは私たちの生活のさまざまな場面で役立')

In [ ]:
## 会話履歴の確認
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='こんにちは！調子はどうだい？'),
  AIMessage(content='こんにちは！私はチャットボットなので、感情はありませんが、お話しすることができます。どのようなお話しをしたいですか？'),
  HumanMessage(content='AIについて教えて'),
  AIMessage(content='AI（人工知能）は、コンピューターシステムが人間のような知的な活動を実行する技術です。AIは、大量のデータを処理し、パターンを認識し、学習することができます。\n\nAIの主なタイプには、以下のようなものがあります：\n\n1. 弱いAI（ナローアイ）：特定のタスクを実行するために設計されたAIです。例えば、音声認識や画像認識などがあります。\n\n2. 強いAI（ジェネラルアイ）：人間のような知的な活動を実行することができるAIです。現在の技術では、まだ完全な強いAIは実現していません。\n\n3. 機械学習：AIの一部として使用される技'),
  HumanMessage(content='小学1年生でもわかりやすく教えて'),
  AIMessage(content='AI（人工知能）は、コンピューターが考えることや学ぶことができるようにする技術です。例えば、お友達の名前を覚えることや、絵を描くことを学ぶことができます。\n\nAIは、たくさんのデータを使って学ぶことができます。例えば、たくさんの写真を見せると、AIは猫や犬の写真を見分けることができます。それは、AIがパターンを認識するからです。\n\nまた、AIは間違いを修正することもできます。例えば、AIが「リンゴ」と言ったときに、実は「バナナ」だった場合、AIは間違いを気づいて修正することができます。\n\nAIは私たちの生活のさまざまな場面で役立')]}

## LangChain Memory

言語モデル自体には、会話履歴内容を保存する機能は持っていません。

LangChainでは、Memoryモジュールを使うことで、会話内容を一時的に保存することが簡単にできます。

![LangChain Memory](https://python.langchain.com/assets/images/memory_diagram-0627c68230aa438f9b5419064d63cbbc.png)

出典: https://python.langchain.com/docs/modules/memory/

LangChain Memoryでは、基本的にシステム内のインメモリに保存するようになっていますが、外部のキャッシュ等のストアに格納することも可能です。

例えば、Redis / Momento Cache / Cloudflare D-1 / Amazon DynamoDB / Firestore / MongoDB 等に保存可能です。

- 参考: [https://js.langchain.com/docs/integrations/chat_memory/redis](https://js.langchain.com/docs/integrations/chat_memory/redis)
